In [19]:
import pandas as pd
df = pd.read_csv('data.txt', encoding='windows-1251')

In [20]:
user_columns = ['DR_Dat', 'DR_Tim', 'DR_NChk', 'DR_NDoc', 
                'DR_Apt', 'DR_NDrugs', 'DR_Kol', 'DR_CZak', 'DR_CRoz', 
                'DR_SDisc', 'DR_TPay', 'DR_CDrugs',  
                'DR_Suppl','DR_CDisc', 'DR_BCDisc', 'DR_TabEmpl',
                'DR_VZak', 'DR_Pos']

df = df[user_columns]

In [21]:
df.columns = ['date', 'time', 'n_chk', 'n_doc', 'apt', 'name_drugs',
       'amnt', 'purch_price', 'retail_price', 'disc', 'pay_type', 'drug_id',
       'supplier', 'disc_id', 'disc_barcode', 'employee', 'vzak', 'pos']

In [22]:
df['disc_barcode'] = df['disc_barcode'].astype('str').replace(r'\.0', '', regex=True)
df['date']= pd.to_datetime(df['date']).dt.strftime('%d.%m.%Y')
df['vzak'] = df['vzak'].astype('str').replace('1', 'offline').replace('2', 'online')
df = df.fillna(0)